In [17]:
%load_ext blackcellmagic

In [3]:
import numpy as np
import scipy.linalg   # SciPy Linear Algebra Library
import math
from pprint import pprint
from numpy import array, zeros, diag, diagflat, dot

# Tiesinių algebrinių lygčių sistemų sprendimas

In [26]:
# Gauso metodas

a = np.array([[2, -1, 0], [-1, 2, -1], [0, -1, 2]])
b = np.array([4, 1, 2])
x = np.linalg.solve(a, b)
for i in range(0, len(x)):
    print("x", i + 1, "=", x[i])
print(x)


x 1 = 4.0
x 2 = 4.0
x 3 = 2.9999999999999996
[4. 4. 3.]


In [16]:
# Choleckio metodas
"""
https://www.quantstart.com/articles/Cholesky-Decomposition-in-Python-and-NumPy
"""


A = scipy.array([[2, 1, 1], [1, 2.5, 1], [1, 1, 3]])
b = scipy.array([3, 5, 0])
L_lower = scipy.linalg.cholesky(A, lower=True)
L_upper = scipy.linalg.cholesky(A, lower=False)

print(L_lower)
L_t = np.transpose(L_lower)
print(L_t)

y = np.linalg.solve(L_lower, b)
print(y)
x = np.linalg.solve(L_upper, y)

print("\n")
for i in range(0, len(x)):
    print("x", i + 1, "=", x[i])
print(x)


[[1.41421356 0.         0.        ]
 [0.70710678 1.41421356 0.        ]
 [0.70710678 0.35355339 1.5411035 ]]
[[1.41421356 0.70710678 0.70710678]
 [0.         1.41421356 0.35355339]
 [0.         0.         1.5411035 ]]
[ 2.12132034  2.47487373 -1.5411035 ]


x 1 = 0.9999999999999999
x 2 = 2.0
x 3 = -1.0000000000000002
[ 1.  2. -1.]


![title](img/1.jpg)

In [43]:
# Tridiagonal matrix algorithm, also known as the Thomas algorithm (Perkelties)

def TDMASolve(a, b, c, d):
    n = len(a)
    ac, bc, cc, dc = map(np.array, (a, b, c, d))
    xc = []
    for j in range(2, n):
        if(bc[j - 1] == 0):
            ier = 1
            return
        ac[j] = ac[j]/bc[j-1]
        bc[j] = bc[j] - ac[j]*cc[j-1]
    if(b[n-1] == 0):
        ier = 1
        return
    for j in range(2, n):
        dc[j] = dc[j] - ac[j]*dc[j-1]
    dc[n-1] = dc[n-1]/bc[n-1]
    for j in range(n-2, -1, -1):
        dc[j] = (dc[j] - cc[j]*dc[j+1])/bc[j]
    return dc

#A = np.array([[2, -2, 0, 0], [1, 13, 3, 0], [0, 1, 9, 2], [0, 0, -2, -15]])

a = np.array([-1,-1])  # A diagonal
b = np.array([2,2,2])  # B diagonal
c = np.array([-1,-1])  # C diagonal

d = np.array([4,1,2])  # known matrix

print(TDMASolve(a, b, c, d))


[2 0 2]


# Iteraciniai sprendimo metodai

In [2]:
#Jakobio metodas
"""
https://www.quantstart.com/articles/Jacobi-Method-in-Python-and-NumPy
"""
def jacobi(A,b,N,x=None):
    """Solves the equation Ax=b via the Jacobi iterative method."""
    # Create an initial guess if needed                                                                                                                                                            
    if x is None:
        x = zeros(len(A[0]))

    # Create a vector of the diagonal elements of A                                                                                                                                                
    # and subtract them from A                                                                                                                                                                     
    D = diag(A)
    R = A - diagflat(D)

    # Iterate for N times                                                                                                                                                                          
    for i in range(N):
        x = (b - dot(R,x)) / D
    return x

A = array([[6,3,0],[3,6,3], [0, 3, 6]])
b = array([0,0,1])
guess = array([0, 0, 0])

sol = jacobi(A,b,N=2,x=None)

print("A:")
pprint(A)

print("b:")
pprint(b)

print("x:")
pprint(sol)

NameError: name 'array' is not defined

In [190]:
#Zeidelio metodas 1
def gauss(A, b, x, n):

    L = np.tril(A)
    U = A - L
    for i in range(n):
        x = np.dot(np.linalg.inv(L), b - np.dot(U, x))
        print (str(i + 1).zfill(3), "\n"),
        print(x)
    return x


A = np.array([[6,3,0], [3,6,3], [0,3,6]])
b = [0,1,0]
x = [0,0,0]

n = 2

print("Guess:")

guess_solution = gauss(A, b, x, n)
print(guess_solution, "\n")

print("Real solution:")

real_solution = np.linalg.solve(A, b)
for i in range(0, len(real_solution)):
    print("x", i + 1, "=", real_solution[i])
print(real_solution)

Guess:
001 

[ 0.          0.16666667 -0.08333333]
002 

[-0.08333333  0.25       -0.125     ]
[-0.08333333  0.25       -0.125     ] 

Real solution:
x 1 = -0.16666666666666666
x 2 = 0.3333333333333333
x 3 = -0.16666666666666666
[-0.16666667  0.33333333 -0.16666667]


In [193]:
# Zeidelio metodas 2
"""
https://github.com/leandroliveira6/GaussSeidelMethod/blob/master/gaussseibelmethod.py
"""
def GaussSeidel(A, b, x0, N, TOL):
    n = len(A)
    k = 1
    while k <= N:
        x = np.zeros(n)
        for i in range(n):
            s = 0.0
            for j in range(i):
                s += A[i, j] * x[j]
            for j in range(i + 1, n):
                s += A[i, j] * x0[j]
            x[i] = (b[i] - s) / A[i, i]
        if np.linalg.norm(x - x0) < TOL:
            return x, k
        k += 1
        x0 = np.copy(x)
    return x0, k


A = np.array([[6, 3, 0], [3, 6, 3], [0, 3, 6]])
b = [0, 1, 0]
guess = [0, 0, 0]

x, n = GaussSeidel(A, b, guess, 2, 0.00000001)
print("Result = {} total {} iterations".format(x.round(4), n))


Result = [-0.0833  0.25   -0.125 ] total 3 iterations
